# 9.1 GAN으로 새로운 패션아이템 생성하기
*GAN을 이용하여 새로운 패션 아이템을 만들어봅니다*

GAN을 설명할때 항상 나오는 명언이 있습니다.
바로 양자 역학에 대한 공으로 노벨 물리학상을 받은
리처드 파인만(Richard Feynman) 교수의
"내가 만들어낼 수 없다면 이해하지 못한 것이다"
(What I cannot create, I do not understand)
라는 문장입니다.
빈 종이가 주어졌을때 처음부터 끝까지 *자신만의
자세한 설명*을 할 수 있어야 진정한 '이해'에
도달한다는 취지에서 말을 한 것이라 생각됩니다.
비슷한 맥락으로 단순히 고양이와 강아지를 구분하는 것과
고양이와 강아지를 그릴 수 있는 것은 이해의 정도에
큰 차이가 있다고 할 수 있습니다.
우리가 지금까지 만든 딥러닝 모델들은 이미지를 구분 하는 정도에 그쳤지만
이제 그림을 그리는 뉴럴넷을 만드는데 도전해보려고 합니다.

Generative Adversarial Networks,
흔히 줄여서 GAN이라 부르고 이안 굿펠로우(Ian Goodfellow)가
2014년 처음 제안한 이 방법론은 한국어로 직역하면
'적대적 생성 신경망'입니다.
왠지 더 어려워보이는 한국어 이름이지만
한 단어씩 뜯어보면 그 의미를 알 수 있습니다.

먼저 GAN은 '생성'을 하는 모델입니다.
기존에 배운 CNN이나 RNN으론 새로운 이미지나 음성을 만들어 낼 수 없었습니다.
GAN은 새로운 이미지나 음성을 '창작'할 수 있도록 고안되었습니다.

두번째로 GAN은 '적대적'으로 학습합니다. 
dversarial 이라는 단어를 사전에 찾아보면
'서로 대립 관계에 있는', 혹은 '적대적인'이라고 나옵니다.
GAN은 가짜 이미지를 생성하는 생성자(Generator)와
이미지의 진위 여부를 판별하는 판별자(Discriminator)가
연달아 학습을 하며 경쟁적으로 (적대적으로) 학습을 하게 됩니다.

마지막으로 GAN은 '신경망'입니다.
Generator와 Discrinimator 모두 뉴럴넷으로 되어있습니다.

GAN을 한마디로 종합하자면 서로 대립하는
두 모델의 경쟁을 통해 학습을 하는 방법론입니다.
얀 르쿤 (Yann LeCun)은 이 아이디어를
"근 20년간의 머신러닝 연구 중 가장 재밌는 아이디어"
라고 했고 앤드류 응(Andrew Ng)을 비롯한 많은 연구자들도
앞으로 딥러닝을 이끌 기술중 하나라고 진단하고 있습니다.

GAN이 이토록 주목받고 미래지향적이라는 평가를 받는
이유중 하나는 바로 비지도 학습을 한다는 것입니다.
GAN은 앞서 배운 오토인코더와 같이 비지도학습을 합니다.
세상에 존재하는 데이터는 기하급수적으로 증가하는 중이고,
대부분의 데이터는 정답(label)이 없습니다.
그렇다고 사람이 모든 데이터를 일일이 가공하기엔 한계가 많습니다. 

## GAN 구현하기

In [1]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.utils import save_image

In [2]:
torch.manual_seed(1)    # reproducible

In [3]:
# Hyper Parameters
EPOCHS = 100
BATCH_SIZE = 100
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("Using Device:", DEVICE)

Using Device: cpu


In [4]:
# Fashion MNIST digits dataset
trainset = datasets.FashionMNIST('./.data',
    train=True,
    download=True,
    transform=transforms.Compose([
       transforms.ToTensor(),
       transforms.Normalize((0.5,), (0.5,))
    ]))
train_loader = torch.utils.data.DataLoader(
    dataset     = trainset,
    batch_size  = BATCH_SIZE,
    shuffle     = True)

In [5]:
# Discriminator
D = nn.Sequential(
        nn.Linear(784, 256),
        nn.LeakyReLU(0.2),
        nn.Linear(256, 256),
        nn.LeakyReLU(0.2),
        nn.Linear(256, 1),
        nn.Sigmoid())

In [1]:
# Generator 
G = nn.Sequential(
        nn.Linear(64, 256),
        nn.ReLU(),
        nn.Linear(256, 256),
        nn.ReLU(),
        nn.Linear(256, 784),
        nn.Tanh())

NameError: name 'nn' is not defined

In [6]:

# Device setting
D = D.to(DEVICE)
G = G.to(DEVICE)

# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = optim.Adam(G.parameters(), lr=0.0002)

In [1]:
total_step = len(train_loader)
for epoch in range(EPOCHS):
    for i, (images, _) in enumerate(train_loader):
        images = images.reshape(BATCH_SIZE, -1).to(DEVICE)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(BATCH_SIZE, 1).to(DEVICE)
        fake_labels = torch.zeros(BATCH_SIZE, 1).to(DEVICE)

        # Train Discriminator

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(BATCH_SIZE, 64).to(DEVICE)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # Train Generator

        # Compute loss with fake images
        z = torch.randn(BATCH_SIZE, 64).to(DEVICE)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, EPOCHS, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))

NameError: name 'train_loader' is not defined

## 참고
본 튜토리얼은 다음 자료를 참고하여 만들어졌습니다.

* [yunjey/pytorch-tutorial](https://github.com/yunjey/pytorch-tutorial) - MIT License